In [ ]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import numpy as np
import optuna
import importlib

sys.path.append(os.path.abspath(".."))

import src.utils.weight_optimizer as wo

In [ ]:
# Load data
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

# MLP
oof_single_1 = np.load("../artifacts/preds/l1/oof_single_1.npy")

# XGB
oof_single_2 = np.load("../artifacts/preds/l1/oof_single_2.npy")

# CB
oof_single_3 = np.load("../artifacts/preds/l1/oof_single_3.npy")

# LGBM
oof_single_4 = np.load("../artifacts/preds/l1/oof_single_4.npy")

In [ ]:
# Tuning
list_v1 = [
    oof_single_1,   # MLP
    oof_single_2,   # XGB
    oof_single_3,   # CB
    oof_single_4    # LGBM
]
importlib.reload(wo)
objective = wo.create_objective(list_v1)

study = wo.run_optuna_search(
    objective,
    n_trials=500,
    n_jobs=1,
    direction="minimize",
    study_name="ens_v1",
    storage=url,
    sampler=optuna.samplers.TPESampler(
        n_startup_trials=100,
        seed=42)
)